In [0]:
from azure.eventhub import EventHubProducerClient, EventData
import json

with open("/dbfs/FileStore/configs/config.json", "r") as f:
    config = json.load(f)
# Read CSV
df = spark.read.option("header", "true").csv("/FileStore/tables/Online_Retail.csv").fillna('')

# Take first 100 rows for testing
#df_small = df.limit()

df = df.repartition(20)

def send_partition(rows):
    producer = EventHubProducerClient.from_connection_string(config["eventhub"]["connection_string"])
    batch = producer.create_batch()
    
    for row in rows:
        event = EventData(json.dumps(row.asDict()))
        try:
            batch.add(event)
        except ValueError:  # batch full
            producer.send_batch(batch)
            batch = producer.create_batch()
            batch.add(event)
    
    if len(batch) > 0:
        producer.send_batch(batch)
    
    producer.close()

df_small.rdd.foreachPartition(send_partition)
